# Imports

In [ ]:
from __future__ import nested_scopes
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('<style>.CodeMirror{font-family: "Courier New";font-size: 12pt;}</style>'))

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import findspark
findspark.init()

In [ ]:
import re
import os
import pandas
pandas.set_option('display.max_rows', None)

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import colors
from matplotlib import rcParams
rcParams['font.sans-serif'] =  'Courier New'
rcParams['font.family'] = 'Courier New'
rcParams['font.size'] = '12'
%matplotlib inline

from IPython.display import display,HTML
import threading
import collections

from IPython.display import display
from ipywidgets import IntProgress,Layout
import time
import threading
import gzip
import pyspark
import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.types import (StructType, StructField, DateType,
    TimestampType, StringType, LongType, IntegerType, DoubleType,FloatType)
from pyspark.sql.functions import to_date, floor
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import lit
import datetime
import time
from pyspark.storagelevel import StorageLevel
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
from pyspark.ml import Pipeline
import pandas
import numpy


import re
import math
from functools import reduce
import json


from pyspark.sql.types import *
from pyspark.sql import functions as F
from datetime import date

In [ ]:
import os

def get_py4jzip():
    spark_home=os.environ['SPARK_HOME']
    py4jzip = !ls {spark_home}/python/lib/py4j*.zip
    return py4jzip[0]

# start analysis cluster and run

In [ ]:
executors_per_node = 5
nodes=3
cores_per_executor=4
task_per_core=4


cache_size=20
total_size=100000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
conf = (SparkConf()
    .set("spark.default.parallelism", "{:d}".format(nodes*executors_per_node))
    .set('spark.executor.instances', '{:d}'.format(nodes*executors_per_node))
    .set('spark.sql.files.maxPartitionBytes', '256m')
    .set('spark.app.name', 'pyspark_final')
    .set('spark.rdd.compress', 'False')
    .set('spark.serializer','org.apache.spark.serializer.KryoSerializer')    
    .set('spark.executor.cores','{:d}'.format(cores_per_executor))
    .set('spark.sql.adaptive.coalescePartitions.initialPartitionNum','{:d}'.format(nodes*executors_per_node*cores_per_executor*task_per_core))
    .set('spark.sql.adaptive.enabled',True)
    .set('spark.sql.adaptive.advisoryPartitionSizeInBytes', '256m')
    .set('spark.executor.memory', '{:d}m'.format(int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
    .set('spark.task.cpus','1')
    .set('spark.driver.memory','128g')
    .set('spark.sql.inMemoryColumnarStorage.compressed','False')
    .set('spark.sql.inMemoryColumnarStorage.batchSize','100000')
#    .set('spark.memory.storageFraction','0.8')
#    .set('spark.memory.fraction','0.7')
        
    .set('spark.sql.execution.arrow.fallback.enabled','True')
    .set('spark.sql.execution.arrow.enabled','True')
    .set('spark.sql.execution.arrow.maxRecordsPerBatch','100000')
    .set("spark.sql.repl.eagerEval.enabled", True)
        
    .set('spark.memory.offHeap.enabled','True')
    .set('spark.memory.offHeap.size','{:d}m'.format(int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
    .set('spark.executor.memoryOverhead','{:d}m'.format(int(math.floor(cache_size*1024/(nodes*executors_per_node)))+3000))
    .set('spark.sql.join.preferSortMergeJoin','False')
#    .set('spark.executor.extraJavaOptions',
#          '-XX:+UseG1GC -XX:+PrintFlagsFinal -XX:+PrintReferenceGC -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCTimeStamps -XX:InitiatingHeapOccupancyPercent=35')
    .set('spark.executor.extraJavaOptions',
          '-XX:+UseParallelGC -XX:+UseParallelOldGC -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCTimeStamps')
    .set('spark.dynamicAllocation.enabled', 'False')
    .set('spark.shuffle.service.enabled', 'False')
    .set('spark.dynamicAllocation.shuffleTracking.enabled', 'Falze')
    .set("spark.sql.legacy.timeParserPolicy","LEGACY")
    .set('spark.executorEnv.PYTHONPATH',f"{os.environ['SPARK_HOME']}/python:{get_py4jzip()}")        
#    .set("spark.sql.session.timeZone", "PST")
     .set("spark.sql.session.timeZone", "Etc/GMT+7")
#     .set("spark.driver.extraJavaOptions","-Duser.timezone=UTC+8")
    
       )

sc = SparkContext(conf=conf,master='yarn')
sc.setLogLevel("ERROR")
spark = SQLContext(sc)
time.sleep(10)


In [ ]:
%%html
<style>
div.output_stderr {
background: #ffdd;
display: none;
}
</style>

# Sparklog

In [ ]:
%run ../sparklog.ipynb

In [ ]:
os.environ["https_proxy"] = "http://10.239.44.250:8080"
os.environ["http_proxy"] = "http://10.239.44.250:8080"

In [ ]:
emonmetric=['emon_cpuutil',
            'emon_cpufreq',
            'emon_instr_retired',
            'emon_ipc',
            'emon_L3 stall',
            'emon_mem stall',
            'emon_uop issue stall',
            'emon_mem_bw',
            'emon_remote ratio',
            'emon_local lat',
            'emon_remote lat',
            'emon_l3 load miss',
            'emon_l3 read miss bw',
            'emon_l3 miss',
            'emon_l2 miss',
            'emon_pcie_bw',
           'emon_cha ia bw',
           'emon_wpq lat',
           'emon_rpq lat',
           'emon_iio comp lat',
           'emon_ocr l3 lat',
           'emon_ocr l2 lat']

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
basedir="gluten/2024_07_03"
appid="application_1713747609775_0279"

last_basedir="gluten/2024_07_02"
last_appid="application_1713747609775_0277"

#snappy
vanilla_basedif="gluten/vanilla/tpch_spark322_sf3t"
vanilla_appid="application_1713747609775_0192"

#gzip
#vanilla_basedif="gluten/vanilla/tpch_spark322_sf3t"
#vanilla_appid="application_1713747609775_0192"


In [ ]:
display(HTML(f'<font size=4><b>Vanilla Traceview</b></font><br><a href=http://sr219:1088/tracing_examples/trace_viewer.html#/tracing/test_data/{vanilla_appid}.json> http://sr219:1088/tracing_examples/trace_viewer.html#/tracing/test_data/{vanilla_appid}.json</a>'))
display(HTML(f'<font size=4><b>last run traceview</b></font><br><a href=http://sr219:1088/tracing_examples/trace_viewer.html#/tracing/test_data/{last_appid}.json> http://sr219:1088/tracing_examples/trace_viewer.html#/tracing/test_data/{last_appid}.json</a>'))

display(HTML(f'<font size=4><b>vanilla spark UI</b></font><br><a href=http://sr219:18080/history/{vanilla_appid}/jobs/> http://sr219:18080/history/{vanilla_appid}/jobs/</a>'))
display(HTML(f'<font size=4><b>last run spark ui</b></font><br><a href=http://sr219:18080/history/{last_appid}/jobs/> http://sr219:18080/history/{last_appid}/jobs/</a>'))


In [ ]:
display(HTML('<a href=#native,-partition-table> 4  native, partition table</a>'))
display(HTML('<a href=#APP-info> 5  APP info</a>'))
display(HTML('<a href=#Compare-to-Last-Run> 6  Compare to Last Run</a>'))
display(HTML('<a href=#Compare-to-Baseline> 7  Compare to Baseline</a>'))
display(HTML('<a href=#config-compare> 8  config compare</a>'))



In [ ]:
disk_prefix=["'nvme0n1'","'nvme1n1'","'nvme2n1'","'nvme3n1'","'nvme4n1'","'nvme5n1'","'nvme6n1'","'nvme7n1'"]
nic_prefix=["'enx0a17179c0c10'","'vethd048c87'","'veth18c3379'"]

# native, partition table

In [ ]:
app=Application_Run(appid,basedir=basedir)
appals=app.analysis['app']['als']

In [ ]:
appals.get_basic_state()

In [ ]:
summary=app.get_summary(show_metric=emonmetric,disk_prefix=disk_prefix,nic_prefix=nic_prefix)
display(summary.style)

In [ ]:
appals.get_app_name()

In [ ]:
shuffle_df, dfx=appals.get_shuffle_stat()

# APP info

In [ ]:
app.generate_trace_view(showemon=True,show_metric=emonmetric,disk_prefix=disk_prefix,nic_prefix=nic_prefix)

In [ ]:
appals=app.analysis['app']['als']

In [ ]:
appals.get_app_info(disk_prefix=disk_prefix,nic_prefix=nic_prefix)

In [ ]:
appals.show_critical_path_time_breakdown()

# Compare to Last Run

In [ ]:
if not last_basedir.startswith("LAST"):
    lastapp=Application_Run(last_appid,basedir=last_basedir)
    output_last=app.compare_app(rapp=lastapp,show_metric=emonmetric,show_queryplan_diff=False,disk_prefix=disk_prefix,nic_prefix=nic_prefix)
    display(HTML(output_last))

# Compare to Baseline

In [ ]:
if vanilla_basedif!="VANILLA_BASEDIR":
    app_base=Application_Run(vanilla_appid,basedir=vanilla_basedif)
    output_base=app.compare_app(rapp=app_base,show_metric=emonmetric,show_queryplan_diff=True,disk_prefix=disk_prefix,nic_prefix=nic_prefix)
    display(HTML(output_base))


# config compare

In [ ]:
if  not last_basedir.startswith("LAST"):
    display(comp_spark_conf(app.analysis['app']['als'],lastapp.analysis['app']['als']))

# convert to HTML

In [ ]:
#%%javascript
#IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
#htmlname=nb_name.replace("ipynb","html")

In [ ]:
#!jupyter nbconvert --to html ./{nb_name} --output html/{htmlname} --template classic

In [ ]:
sc.stop()